In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.Resize((28, 28)),
                                       transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

In [3]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [4]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? True


In [5]:
class Model3(nn.Module):
    def __init__(self):
        super(Model3, self).__init__()

        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=0),
            nn.BatchNorm2d(10),
            nn.ReLU()
        )  # INPUT = 28x28 | OUTPUT = 26x26 | RF = 3

        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(3, 3), padding=0),
            nn.BatchNorm2d(10),
            nn.ReLU()
        )  # INPUT = 26x26 | OUTPUT = 24x24 | RF = 5

        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(3, 3), padding=0),
            nn.BatchNorm2d(10),
            nn.ReLU()
        )  # INPUT = 24x24 | OUTPUT = 22x22 | RF = 7

        self.pool1 = nn.MaxPool2d(2, 2)  # INPUT = 22x22 | OUTPUT = 11x11 | RF = 8

        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(3, 3), padding=0),
            nn.BatchNorm2d(10),
            nn.ReLU()
        )  # INPUT = 11x11 | OUTPUT = 9x9 | RF = 12

        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0),
            nn.BatchNorm2d(20),
            nn.ReLU()
        )  # INPUT = 9x9 | OUTPUT = 7x7 | RF = 16

        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(3, 3), padding=0),
            nn.BatchNorm2d(10),
            nn.ReLU()
        )  # INPUT = 7x7 | OUTPUT = 5x5 | RF = 20

        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(3, 3), padding=0),
        )  # INPUT = 5x5 | OUTPUT = 3x3 | RF = 24

        self.gap = nn.Sequential(
            nn.AvgPool2d(3)
        )  # INPUT = 3x3 | OUTPUT = 1x1 | RF = 28

        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.dropout(x)
        x = self.convblock6(x)
        x = self.convblock7(x)
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)


In [6]:
%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Model3().to(device)
summary(model, input_size=(1, 28, 28))

Note: you may need to restart the kernel to use updated packages.
cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
              ReLU-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 10, 24, 24]             910
       BatchNorm2d-5           [-1, 10, 24, 24]              20
              ReLU-6           [-1, 10, 24, 24]               0
            Conv2d-7           [-1, 10, 22, 22]             910
       BatchNorm2d-8           [-1, 10, 22, 22]              20
              ReLU-9           [-1, 10, 22, 22]               0
        MaxPool2d-10           [-1, 10, 11, 11]               0
           Conv2d-11             [-1, 10, 9, 9]             910
      BatchNorm2d-12             [-1, 10, 9, 9]              20
             ReLU-13            

In [7]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))
    return test_loss,test_acc

In [8]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

model =  Model3().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01) #replaced SGD with Adam
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    #test(model, device, test_loader)
    test_loss, _ = test(model, device, test_loader)
    scheduler.step(test_loss)

/opt/conda/envs/pytorch/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


EPOCH: 0


Loss=0.06316276639699936 Batch_id=468 Accuracy=95.75: 100%|██████████| 469/469 [00:08<00:00, 58.35it/s] 



Test set: Average loss: 0.0532, Accuracy: 9816/10000 (98.16%)

EPOCH: 1


Loss=0.01732787676155567 Batch_id=468 Accuracy=98.31: 100%|██████████| 469/469 [00:07<00:00, 60.94it/s]  


...


Test set: Average loss: 0.0149, Accuracy: 9956/10000 (99.56%)

EPOCH: 13


Loss=0.031765781342983246 Batch_id=468 Accuracy=99.67: 100%|██████████| 469/469 [00:07<00:00, 62.25it/s]  



Test set: Average loss: 0.0163, Accuracy: 9946/10000 (99.46%)

EPOCH: 14


Loss=0.02382434904575348 Batch_id=468 Accuracy=99.71: 100%|██████████| 469/469 [00:07<00:00, 62.00it/s]   



Test set: Average loss: 0.0158, Accuracy: 9955/10000 (99.55%)



**Target:**

Introduce Scheduler

Keep the epochs within 15

Reach test accuracy greater than 99.4

**Results:**

Parameters: 7,510

Best Training Accuracy: 99.71

Best Test Accuracy: 99.56

**Analysis:**

Parameters count has been kept under 8000

Model achieved test accuracy greater than 99.4 from 10th epoch